# Get mlrpt working with the saved data

In [1]:
library(tidyverse)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
library(lubridate)


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



Need a command string in the form of `mlrpt $startT-$stopT (time HHMM in UTC) -t $durationMin`

In [3]:
latest_data <- list.files(pattern="satellite_data_*") %>% sort(decreasing = TRUE) %>% pluck(1)

In [4]:
satellite_data_df <- readRDS(latest_data)

Convert to satellite passes each day

Get the current time in UTC

In [5]:
(current_time_utc <- Sys.time() %>% .POSIXct("GMT"))

[1] "2019-08-28 15:03:22 GMT"

In [6]:
satellite_data_df <- satellite_data_df %>% filter(startUTC > current_time_utc)

In [7]:
satellite_data_df

startAz,startAzCompass,startUTC,maxAz,maxAzCompass,maxEl,maxUTC,endAz,endAzCompass,endUTC,startDate,startTime,endDate,endTime,localStartTime,duration
<dbl>,<chr>,<dttm>,<dbl>,<chr>,<dbl>,<int>,<dbl>,<chr>,<dttm>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
165.70,S,2019-08-28 18:31:10,77.09,E,75.69,1567017535,350.30,N,2019-08-28 18:46:30,2019-08-28,18:31,2019-08-28,18:46,20:30,15
20.87,NNE,2019-08-29 05:47:15,106.98,ESE,67.14,1567058095,188.32,S,2019-08-29 06:02:35,2019-08-29,05:47,2019-08-29,06:02,07:46,15
159.87,SSE,2019-08-29 18:11:20,80.30,E,46.64,1567102740,1.17,N,2019-08-29 18:26:25,2019-08-29,18:11,2019-08-29,18:26,20:10,15
31.63,NE,2019-08-30 05:27:40,107.29,ESE,41.89,1567143305,182.54,S,2019-08-30 05:42:30,2019-08-30,05:27,2019-08-30,05:42,07:26,15
153.36,SSE,2019-08-30 17:51:45,83.50,E,29.11,1567187940,12.35,N,2019-08-30 18:06:10,2019-08-30,17:51,2019-08-30,18:06,19:50,15
183.14,S,2019-08-30 19:31:55,249.68,WSW,26.68,1567193940,316.76,NW,2019-08-30 19:45:55,2019-08-30,19:31,2019-08-30,19:45,21:30,14
42.63,NE,2019-08-31 05:08:15,109.90,ESE,26.86,1567228520,176.79,S,2019-08-31 05:22:25,2019-08-31,05:08,2019-08-31,05:22,07:07,14
347.90,N,2019-08-31 06:48:05,276.92,W,29.21,1567234520,206.75,SSW,2019-08-31 07:02:35,2019-08-31,06:48,2019-08-31,07:02,08:47,14
177.28,S,2019-08-31 19:11:45,252.09,WSW,41.65,1567279155,328.15,NW,2019-08-31 19:26:35,2019-08-31,19:11,2019-08-31,19:26,21:10,15


In [12]:
paste0('systemd-run --user --no-block --unit=\"mlrpt_', satellite_data_df$startDate, "_",
       satellite_data_df$localStartTime,
       '\" --on-calendar=\"', satellite_data_df$startDate," ", satellite_data_df$localStartTime,
       '\" /usr/local/bin/mlrpt \"', satellite_data_df$startTime,"-", satellite_data_df$endTime,
       '\" -t ', satellite_data_df$duration)

[1] "systemd-run --user --no-block --unit=\"mlrpt_2019-08-28_20:30\" --on-calendar=\"2019-08-28 20:30\" /usr/local/bin/mlrpt \"18:31-18:46\" -t 15"
 [2] "systemd-run --user --no-block --unit=\"mlrpt_2019-08-29_07:46\" --on-calendar=\"2019-08-29 07:46\" /usr/local/bin/mlrpt \"05:47-06:02\" -t 15"
 [3] "systemd-run --user --no-block --unit=\"mlrpt_2019-08-29_20:10\" --on-calendar=\"2019-08-29 20:10\" /usr/local/bin/mlrpt \"18:11-18:26\" -t 15"
 [4] "systemd-run --user --no-block --unit=\"mlrpt_2019-08-30_07:26\" --on-calendar=\"2019-08-30 07:26\" /usr/local/bin/mlrpt \"05:27-05:42\" -t 15"
 [5] "systemd-run --user --no-block --unit=\"mlrpt_2019-08-30_19:50\" --on-calendar=\"2019-08-30 19:50\" /usr/local/bin/mlrpt \"17:51-18:06\" -t 15"
 [6] "systemd-run --user --no-block --unit=\"mlrpt_2019-08-30_21:30\" --on-calendar=\"2019-08-30 21:30\" /usr/local/bin/mlrpt \"19:31-19:45\" -t 14"
 [7] "systemd-run --user --no-block --unit=\"mlrpt_2019-08-31_07:07\" --on-calendar=\"2019-08-31 07:07\" /usr/local/bin/mlrpt \"05:08-05:22\" -t 14"
 [8] "systemd-run --user --no-block --unit=\"mlrpt_2019-08-31_08:47\" --on-calendar=\"2019-08-31 08:47\" /usr/local/bin/mlrpt \"06:48-07:02\" -t 14"
 [9] "systemd-run --user --no-block --unit=\"mlrpt_2019-08-31_21:10\" --on-calendar=\"2019-08-31 21:10\" /usr/local/bin/mlrpt \"19:11-19:26\" -t 15"
[10] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-01_08:26\" --on-calendar=\"2019-09-01 08:26\" /usr/local/bin/mlrpt \"06:27-06:42\" -t 15"
[11] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-01_20:50\" --on-calendar=\"2019-09-01 20:50\" /usr/local/bin/mlrpt \"18:51-19:07\" -t 16"
[12] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-02_08:06\" --on-calendar=\"2019-09-02 08:06\" /usr/local/bin/mlrpt \"06:07-06:23\" -t 16"
[13] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-02_20:30\" --on-calendar=\"2019-09-02 20:30\" /usr/local/bin/mlrpt \"18:31-18:47\" -t 16"
[14] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-03_07:46\" --on-calendar=\"2019-09-03 07:46\" /usr/local/bin/mlrpt \"05:47-06:03\" -t 16"
[15] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-03_20:11\" --on-calendar=\"2019-09-03 20:11\" /usr/local/bin/mlrpt \"18:12-18:27\" -t 15"
[16] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-04_07:27\" --on-calendar=\"2019-09-04 07:27\" /usr/local/bin/mlrpt \"05:28-05:43\" -t 15"
[17] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-04_19:51\" --on-calendar=\"2019-09-04 19:51\" /usr/local/bin/mlrpt \"17:52-18:06\" -t 14"
[18] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-04_21:31\" --on-calendar=\"2019-09-04 21:31\" /usr/local/bin/mlrpt \"19:32-19:46\" -t 14"
[19] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-05_07:07\" --on-calendar=\"2019-09-05 07:07\" /usr/local/bin/mlrpt \"05:08-05:23\" -t 15"
[20] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-05_08:47\" --on-calendar=\"2019-09-05 08:47\" /usr/local/bin/mlrpt \"06:48-07:03\" -t 15"
[21] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-05_21:11\" --on-calendar=\"2019-09-05 21:11\" /usr/local/bin/mlrpt \"19:12-19:27\" -t 15"
[22] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-06_08:27\" --on-calendar=\"2019-09-06 08:27\" /usr/local/bin/mlrpt \"06:28-06:43\" -t 15"
[23] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-06_20:51\" --on-calendar=\"2019-09-06 20:51\" /usr/local/bin/mlrpt \"18:52-19:07\" -t 15"
[24] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-07_08:07\" --on-calendar=\"2019-09-07 08:07\" /usr/local/bin/mlrpt \"06:08-06:23\" -t 15"

In [13]:
read_delim("timers.txt")

ERROR: Error in read_delim("timers.txt"): argument "delim" is missing, with no default
